In [2]:
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import tensorflow as tf
from keras.models import model_from_json
from keras.models import load_model
#from deeplabv3 import relu6, BilinearUpsampling
from keras.engine.topology import get_source_inputs
from keras.optimizers import Adam
from keras import backend as K
from keras.applications import imagenet_utils
from keras.utils import conv_utils, multi_gpu_model
from keras.utils.data_utils import get_file
import os
import numpy as np

Using TensorFlow backend.


In [3]:
def read_paths(dataset_path, list_path):
    imagepaths = []
    labelpaths = []
    for path in tqdm(open(list_path)):
        md5 = path.split('.')[0]
        imagepaths.append(dataset_path + "images/%s.jpg"%md5)
        labelpaths.append(dataset_path + "spline_labels/%s.json"%md5)
    return imagepaths, labelpaths

# detect points for one image
def detect_points(img, model, shape):
    img = cv2.resize(img, (shape,shape))
    img = np.expand_dims(img,axis=0)
    points_logist = model.predict(img)
    points = np.argmax(points_logist, axis=-1)
    return points[0]

In [4]:
import os
from deeplabv3 import relu6, BilinearUpsampling

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def config_keras_backend(gpu_memory_fraction):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = gpu_memory_fraction
    sess = tf.Session(config=config)
    K.set_session(sess)

GPU_MEMORY_FRACTION = 0.1
config_keras_backend(GPU_MEMORY_FRACTION)
model = load_model('deeplab_0614.h5', custom_objects={'relu6':relu6, 'BilinearUpsampling':BilinearUpsampling})
# use val set to show the result example
val_set_path = '/data/mc_data/MLDC/data/val/'
val_list_path = '/data/mc_data/MLDC/data/val/list.txt'
val_image_paths, val_label_paths = read_paths(val_set_path, val_list_path)

/home/mc16/.local/lib/python2.7/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
2015it [00:00, 298166.26it/s]


In [5]:
# show the detected points on a valid image
DATA_SHAPE = 224
#train_labels = np.load('/home/mc16/pre_data/train_label_%s.npy'%DATA_SHAPE)
val_labels = np.load('/home/mc16/pre_data/val_label_%s.npy'%DATA_SHAPE)
#train_images = np.load('/home/mc16/pre_data/train_image_%s.npy'%DATA_SHAPE)
val_images = np.load('/home/mc16/pre_data/val_image_%s.npy'%DATA_SHAPE)

test_index = random.randint(0, len(val_labels))
test_label = val_labels[test_index,:,:,1]
test_img = plt.imread(val_image_paths[test_index])
test_img = cv2.resize(test_img,(DATA_SHAPE,DATA_SHAPE))
test_points = detect_points(test_img, model, DATA_SHAPE)
fig, axs = plt.subplots(1, 3, figsize=(30,30)) 
axs[0].imshow(test_img)
axs[1].imshow(test_label,cmap='gray')
axs[2].imshow(test_points,cmap='gray')
plt.show()

IOError: [Errno 2] No such file or directory: '/home/mc16/pre_data/val_label_224.npy'